In [1]:
# 使用tushare 下载数据

import tushare as ts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# 设置token
ts.set_token('4167d168dd2c182108319b0f5fc93d66b21d5ccf2baa2af2a522b80d')
pro = ts.pro_api()



In [4]:
# 获取中国基准利率
df = pro.shibor(start_date='20100101', end_date='20200101')
df = df[['date', 'on', '1w', '2w', '1m', '3m', '6m', '9m', '1y']]
# df = df.rename(columns={'on': 'ON', '1w': '1W', '2w': '2W', '1m': '1M', '3m': '3M', '6m': '6M', '9m': '9M', '1y': '1Y'})
df = df.set_index('date')
df = df.astype('float')
df = df.sort_index()
df = df / 100
# save
df.to_csv('../data/train/base_rate.csv')

In [5]:
# 获取中国基准利率
df = pro.shibor(start_date='20200101', end_date='20230101')
df = df[['date', 'on', '1w', '2w', '1m', '3m', '6m', '9m', '1y']]
# df = df.rename(columns={'on': 'ON', '1w': '1W', '2w': '2W', '1m': '1M', '3m': '3M', '6m': '6M', '9m': '9M', '1y': '1Y'})
df = df.set_index('date')
df = df.astype('float')
df = df.sort_index()
df = df / 100
# save
df.to_csv('../data/test/base_rate.csv')

In [6]:
df.head()

,on,1w,2w,1m,3m,6m,9m,1y
date,,,,,,,,
20200102,0.014436,0.02433,0.02334,0.028975,0.02994,0.03019,0.03052,0.03086
20200103,0.011620,0.02349,0.02190,0.028420,0.02960,0.02995,0.03028,0.03071
20200106,0.010030,0.02219,0.02091,0.027470,0.02920,0.02960,0.03005,0.03050
20200107,0.012150,0.02259,0.02176,0.027000,0.02899,0.02943,0.02992,0.03040
20200108,0.015530,0.02320,0.02160,0.026380,0.02881,0.02930,0.02983,0.03032


# DJIA 未成功，是用的雅虎财经在local下载再上传的

In [1]:
# download DJIA
# 使用tushare 下载数据

import tushare as ts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# 设置token
ts.set_token('4167d168dd2c182108319b0f5fc93d66b21d5ccf2baa2af2a522b80d')
pro = ts.pro_api()

# 获取道琼斯指数
df = pro.index_daily(ts_code='000001.SH', start_date='20100101', end_date='20200101')
df = df[['trade_date', 'close']]
df = df.rename(columns={'trade_date': 'date', 'close': 'DJIA'})
df = df.set_index('date')
df = df.astype('float')
df = df.sort_index()
# save
df.to_csv('../data/train/DJIA.csv')





In [8]:
data_df = pd.read_csv('../DRL_FinRL_model/datasets/DJI.csv')

In [10]:
data_df.head()

,index,Open,High,Low,Close,Adj Close,Volume
0,0,28638.970703,28872.800781,28627.769531,28868.800781,28868.800781,251820000
1,1,28553.330078,28716.310547,28500.359375,28634.880859,28634.880859,239590000
2,2,28465.500000,28708.019531,28418.630859,28703.380859,28703.380859,252760000
3,3,28639.179688,28685.500000,28565.279297,28583.679688,28583.679688,258900000
4,4,28556.140625,28866.179688,28522.509766,28745.089844,28745.089844,291750000


In [9]:
data_df = data_df.reset_index()
try:
    # convert the column names to standardized names
    data_df.columns = [
        "date",
        "open",
        "high",
        "low",
        "close",
        "adjcp",
        "volume",
        "tic",
    ]
    # use adjusted close price instead of close price
    data_df["close"] = data_df["adjcp"]
    # drop the adjusted close price column
    data_df = data_df.drop(labels="adjcp", axis=1)
except NotImplementedError:
    print("the features are not supported currently")
# create day of the week column (monday = 0)
data_df["day"] = data_df["date"].dt.dayofweek
# convert date to standard string format, easy to filter
data_df["date"] = data_df.date.apply(lambda x: x.strftime("%Y-%m-%d"))
# drop missing data
data_df = data_df.dropna()
data_df = data_df.reset_index(drop=True)
print("Shape of DataFrame: ", data_df.shape)
# print("Display DataFrame: ", data_df.head())

data_df = data_df.sort_values(by=["date", "tic"]).reset_index(drop=True)

data_df.sample(5)

ValueError: Length mismatch: Expected axis has 7 elements, new values have 8 elements

# 上证50

In [6]:
# download 上证50
# 使用tushare 下载数据

import tushare as ts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# 设置token
ts.set_token('4167d168dd2c182108319b0f5fc93d66b21d5ccf2baa2af2a522b80d')
pro = ts.pro_api()

# 获取上证50指数
df = pro.index_daily(ts_code='000016.SH', start_date='20160101', end_date='20230101')
# columns 'date', 'open', 'high', 'low', 'close', 'volume', 'tic', 'day'
df.rename(columns={'trade_date': 'date', 'vol': 'volume', 'ts_code': 'tic'}, inplace=True)
df.drop(columns=['pre_close', 'change', 'pct_chg','amount'], inplace=True)
# day
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df['day'] = df['date'].dt.dayofweek
# sort
df = df.sort_values(by=["date", "tic"]).reset_index(drop=True)
# save
df.to_csv('../DRL_FinRL_model/datasets/SSE_50_Index.csv')